# Matrices Tridiagonales

In [30]:
import numpy as np
from scipy.linalg import solve_banded
import matplotlib.pyplot as plt

# Fundamento Teorico 

Sea la siguiente sistemas de ecuaciones que usualmente se presenta en Física

\begin{equation}
\begin{pmatrix}
a_{00} & a_{01} & 0 & 0\\
a_{10} & a_ {11} & a_{12} & 0\\
0 & a_{21} & a_{22} & a_{23} \\
0 & 0 & a_{32} & a_{33} 
\end{pmatrix} 
\begin{pmatrix}
x_0 \\x_1 \\ x_2 \\ x_3  
\end{pmatrix} 
= \begin{pmatrix}
v_0 \\v_1 \\ v_2 \\ v_3 
\end{pmatrix}
\end{equation}   

El objetivo es reducirlo por eliminación forward: 

\begin{equation}
\begin{pmatrix}
1 & a_{01} & 0 & 0\\
0 & 1 & a_{12} & 0\\
0 & 0 & 1 & a_{23} \\
0 & 0 & 0 & 1 
\end{pmatrix} 
\begin{pmatrix}
x_0 \\x_1 \\ x_2 \\ x_3 
\end{pmatrix} 
= \begin{pmatrix}
v_0 \\v_1 \\ v_2 \\ v_3 
\end{pmatrix}
\end{equation} 

y hacer la sustitución Backward 

\begin{align}
x_3 &= v_3 \\
x_2 &= v_2 - a_{23} x_3 \\
x_1 &= v_1 - a_{12} x_2 \\
x_0 &= v_0 - a_{01} x_1 
\end{align}

Como puede notarse esto es conocido de la resolución general de matrices pero la diferencia es que hay menos pasos en la eliminación forward y bacward sustitución. 

### Implementando la eliminacion gaussiana para matrices tridiagonales  ( forward eliminación  con back sustitutión ) 

In [31]:
def naive_gauss(a,b):
    '''Funcion que retorna un array "x" despues de hacer una eliminacion de Gauss sin pivoteo'''
    
    n = a.shape[0] # la dimension de mi matriz 
    # https://numpy.org/doc/1.18/reference/generated/numpy.ndarray.html
    # estamos creando un matriz columna 
    x = np.ndarray( shape=(n,1), dtype = np.float64 )
    
    # el -1 es para indicar que recorreremos n-1 filas 
    # ya que al final la ultima fila no sera necesario 
    # hacer un eliminacion 
    for k in range(0,n-1):
        
        # ###### ELIMINACION DE GAUSS (FORWARD) ############
        # este indice recorrera las filas 
        # haciendo ceros 
        for i in range(k+1,n):
            
            # esta variable se crea con el fin de no calcularla 
            # muchas veces dentro del bucle del "j"
            xmult = a[i][k]/a[k][k] 
            # en principio a[i][k] es 0 como esperamos pero 
            a[i][k] = xmult  
            
            # este indice hara el seguimiento de los 
            # los cambios a los demas elementos de la fila
            for j in range(k+1,len(a)):
                # hacemos los cambios en fila completa
                a[i][j] -= xmult*a[k][j]
            
            # hacemos por separado el cambio al elemento de "b"
            b[i] -= xmult*b[k]
    
    # ###### SUBSTITUCION INVERSA(BACKWARD) ######## 
    # espero se entienda el por que del -1 
    x[n-1] = b[n-1]/a[n-1][n-1] # comenzamos por el ultimo
       
    # luego hacemos el algoritmos de adelante hacia atras
    # "i" en general siempre se referira a filas y "j" a columnas
    
    # Espero se entienda el por que de "n-2"
    # es debido a la funcion range(start,final,pasos)
    for i in range(n-2,-1,-1):
        
        # sera que hara cuenta de todo lo que se restara a b[i]
        # lo ponemos aqui debido a no llamarla demasiadas veces
        summ = b[i] 
        
        for j in range(i+1,n):
            summ -= a[i][j]*x[j]
        
        # finalmente asignamos
        x[i] = summ/a[i][i]
    
    return x

### Probando la función naive_gauss

In [214]:
print(" ######## Probando en un ejemplo particular ############## \n")
# con el dtype me aseguro que todos seran floats de 64 bits        
M = np.array( [[2,1,0,0],[3,4,-5,0],[0,-4,3,5],[0,0,1,3]] , dtype= np.float64 )
N = np.array( [[3],[4],[7],[9]] , dtype= np.float64 )

display(M)

for i in range(0,len(N)):
    print(f'{M[i,0]}*x1 + {M[i,1]}*x2 + {M[i,2]}*x3 + {M[i,3]}*x4  = {N[i,0]}')
    
X = naive_gauss(M.copy(),N.copy())
print("Respuesta es : \n")
for i in range(0,len(X)):
    print(f'x{i+1} es : {X[i]}')

 ######## Probando en un ejemplo particular ############## 



array([[ 2.,  1.,  0.,  0.],
       [ 3.,  4., -5.,  0.],
       [ 0., -4.,  3.,  5.],
       [ 0.,  0.,  1.,  3.]])

2.0*x1 + 1.0*x2 + 0.0*x3 + 0.0*x4  = 3.0
3.0*x1 + 4.0*x2 + -5.0*x3 + 0.0*x4  = 4.0
0.0*x1 + -4.0*x2 + 3.0*x3 + 5.0*x4  = 7.0
0.0*x1 + 0.0*x2 + 1.0*x3 + 3.0*x4  = 9.0
Respuesta es : 

x1 es : [0.28]
x2 es : [2.44]
x3 es : [1.32]
x4 es : [2.56]


### comprobando con scipy Ver https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_banded.html 

In [182]:
print("\n###### Comprobando el resultado ######")

### Creamos la matriz diagonal 

d = [2,4,3,3]
d_u = [1,-5,5]
d_d = [3,-4,1]

M = np.diag(d) + np.diag(d_u, 1) + np.diag(d_d, -1)

display(M)


u = 1
l = 1
num = len(M)
ab = np.zeros((l + u + 1, num))

for i in range(0,num):
    for j in range(0,num):
        if (u+i-j >= 0 and u+i-j < l + u + 1):
            ab[u + i - j, j] = M[i,j]
             
display(ab)


N = np.array( [[3],[4],[7],[9]] , dtype= np.float64 )

result = scipy.linalg.solve_banded((1, 1), ab, N)

print("Respuesta es : \n")
for i in range(0,len(result)):
    print(f'x{i+1} es : {result[i]}')


###### Comprobando el resultado ######


array([[ 2,  1,  0,  0],
       [ 3,  4, -5,  0],
       [ 0, -4,  3,  5],
       [ 0,  0,  1,  3]])

array([[ 0.,  1., -5.,  5.],
       [ 2.,  4.,  3.,  3.],
       [ 3., -4.,  1.,  0.]])

Respuesta es : 

x1 es : [0.28]
x2 es : [2.44]
x3 es : [1.32]
x4 es : [2.56]


# Algoritmo para matrices banded

In [180]:
def naive_gauss_banded(a,b,up,down):
    '''Funcion que retorna un array "x" despues de hacer una eliminacion de Gauss sin pivoteo'''
    
    n = a.shape[0] # la dimension de mi matriz 
    # https://numpy.org/doc/1.18/reference/generated/numpy.ndarray.html
    # estamos creando un matriz columna 
    x = np.ndarray( shape=(n,1), dtype = np.float64 )
    
    # el -1 es para indicar que recorreremos n-1 filas 
    # ya que al final la ultima fila no sera necesario 
    # hacer un eliminacion 
    for k in range(0,n-1):
        
        # ###### ELIMINACION DE GAUSS (FORWARD) ############
        # este indice recorrera las filas 
        # haciendo ceros 
        for i in range(k+1,n):
            
            if i> k+down and j> k+up:
                break
            
            # esta variable se crea con el fin de no calcularla 
            # muchas veces dentro del bucle del "j"
            xmult = a[i][k]/a[k][k] 
            # en principio a[i][k] es 0 como esperamos pero 
            a[i][k] = xmult  
            
            # este indice hara el seguimiento de los 
            # los cambios a los demas elementos de la fila
            for j in range(k+1,len(a)):
                # hacemos los cambios en fila completa
                if i> k+down and j> k+up:
                    break
                a[i][j] -= xmult*a[k][j]
            
            # hacemos por separado el cambio al elemento de "b"
            b[i] -= xmult*b[k]
    
    # ###### SUBSTITUCION INVERSA(BACKWARD) ######## 
    # espero se entienda el por que del -1 
    x[n-1] = b[n-1]/a[n-1][n-1] # comenzamos por el ultimo
       
    # luego hacemos el algoritmos de adelante hacia atras
    # "i" en general siempre se referira a filas y "j" a columnas
    
    # Espero se entienda el por que de "n-2"
    # es debido a la funcion range(start,final,pasos)
    for i in range(n-2,-1,-1):
        
        
        # sera que hara cuenta de todo lo que se restara a b[i]
        # lo ponemos aqui debido a no llamarla demasiadas veces
        summ = b[i] 
        
        for j in range(i+1,n):
            
            if j> i+up:
                break
                
            summ -= a[i][j]*x[j]
        
        # finalmente asignamos
        x[i] = summ/a[i][i]
    
    return x

# Resolviendo un problema de Computational physics Mark Newman ( Exercise 6.7 cadena de resistores ) 

<img src="img/616.png">

\begin{align}
3V_1 - V_2 - V_3 &= V_{+} \\
-V_1 + 4V_2 - V_3 - V_4 &= V_{+} \\
\vdots \\
-V_{i-2} - V_{i-1} + 4V_{i} - V_{i+1} - V_{i+2} &= 0 \\
\vdots \\
-V_{N-3} - V_{N-2} + 4V_{N-1} - V{V_N} &= 0 \\
-V_{N-2} - V_{N-1} + 3V_N & = 00 
\end{align}

\begin{align}
\begin{pmatrix}
3 & -1 & -1 & 0 & 0 & 0 & 0\\
-1 & +4 & -1 & -1 & 0 & 0 & 0\\
-1 & -1 & +4 & -1 & -1 & 0 & 0 \\
0& -1 & -1 & +4 & -1 & -1 & 0  \\
0 & 0 & -1 & -1 & +4 & -1 & -1\\
0 & 0 & 0 & -1 & -1 & +4 & -1\\
0 & 0& 0 & 0 & -1 & -1 & 3 
\end{pmatrix}  
\end{align}

In [218]:
def matriz_resistor(n):
    '''
    funcion que crea la matriz que resuelve el problema 
    de la resistencia con N nodos y tambien la matriz b 
    '''
    A = np.zeros(shape=(n,n), dtype=np.float64)
    # llenamos las dos primeras filas y dos primeras columnas 
    # que son particulares 
    A[0][0:4] = [3,-1,-1,0]
    A[1][0:4] = [-1,4,-1,-1]
    A[n-2][n-4:n] = [-1,-1,4,-1]
    A[n-1][n-4:n] = [0,-1,-1,3]
    # ese rango es debido a que llenamos ya la fila 0,1,n,n-1
    for i in range(2,n-2):
        for j in range(0,n):
            if j == i :
                A[i,j] = 4
                A[i,j-1] = -1
                A[i,j-2] = -1
                A[i,j+1] = -1
                A[i,j+2] = -1 
                
    # Definimos tambien el vector W 
    cola_inicial = [5,5]
    cola_final = [0]*(n-2)
    
    b = np.array([cola_inicial + cola_final],dtype=np.float64) ## cuidado con esto 
    
    
    return A ,b.T

# Para N = 10 

In [224]:
A,b = matriz_resistor(10)
display(A,b)
V = naive_gauss_banded(A.copy(),b.copy(),2,2)

print("Respuesta es : \n")
for i in range(0,len(V)):
    print(f'V{i+1} es : {V[i]}')

array([[ 3., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  4., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1., -1.,  4., -1., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1., -1.,  4., -1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1., -1.,  4., -1., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1., -1.,  4., -1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -1.,  4., -1., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1., -1.,  4., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  4., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  3.]])

array([[5.],
       [5.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

Respuesta es : 

V1 es : [4.12573674]
V2 es : [3.9194499]
V3 es : [3.45776031]
V4 es : [3.09430255]
V5 es : [2.69155206]
V6 es : [2.30844794]
V7 es : [1.90569745]
V8 es : [1.54223969]
V9 es : [1.0805501]
V10 es : [0.87426326]


In [225]:
# comprobando con numpy 
V = np.linalg.solve(A,b)
print("Respuesta es : \n")
for i in range(0,len(V)):
    print(f'V{i+1} es : {V[i]}')

Respuesta es : 

V1 es : [4.12573674]
V2 es : [3.9194499]
V3 es : [3.45776031]
V4 es : [3.09430255]
V5 es : [2.69155206]
V6 es : [2.30844794]
V7 es : [1.90569745]
V8 es : [1.54223969]
V9 es : [1.0805501]
V10 es : [0.87426326]


# N = 10 000

In [231]:
# Con mi codigo 
A,b = matriz_resistor(10000)
V = naive_gauss_banded(A.copy(),b.copy(),2,2)

print("Respuesta es : \n")
for i in range(0,len(V)):
    print(f'V{i+1} es : {V[i]}')

Respuesta es : 

V1 es : [4.99888228]
V2 es : [4.99861842]
V3 es : [4.99802841]
V4 es : [4.99756299]
V5 es : [4.99704997]
V6 es : [4.99655514]
V7 es : [4.99605336]
V8 es : [4.99555423]
V9 es : [4.99505409]
V10 es : [4.99455433]
V11 es : [4.99405443]
V12 es : [4.99355459]
V13 es : [4.99305472]
V14 es : [4.99255486]
V15 es : [4.992055]
V16 es : [4.99155514]
V17 es : [4.99105528]
V18 es : [4.99055542]
V19 es : [4.99005556]
V20 es : [4.9895557]
V21 es : [4.98905584]
V22 es : [4.98855598]
V23 es : [4.98805612]
V24 es : [4.98755626]
V25 es : [4.9870564]
V26 es : [4.98655654]
V27 es : [4.98605667]
V28 es : [4.98555681]
V29 es : [4.98505695]
V30 es : [4.98455709]
V31 es : [4.98405723]
V32 es : [4.98355737]
V33 es : [4.98305751]
V34 es : [4.98255765]
V35 es : [4.98205779]
V36 es : [4.98155793]
V37 es : [4.98105807]
V38 es : [4.98055821]
V39 es : [4.98005835]
V40 es : [4.97955849]
V41 es : [4.97905863]
V42 es : [4.97855877]
V43 es : [4.97805891]
V44 es : [4.97755904]
V45 es : [4.97705918]
V46 es

V521 es : [4.73912554]
V522 es : [4.73862568]
V523 es : [4.73812582]
V524 es : [4.73762596]
V525 es : [4.7371261]
V526 es : [4.73662624]
V527 es : [4.73612638]
V528 es : [4.73562652]
V529 es : [4.73512666]
V530 es : [4.7346268]
V531 es : [4.73412693]
V532 es : [4.73362707]
V533 es : [4.73312721]
V534 es : [4.73262735]
V535 es : [4.73212749]
V536 es : [4.73162763]
V537 es : [4.73112777]
V538 es : [4.73062791]
V539 es : [4.73012805]
V540 es : [4.72962819]
V541 es : [4.72912833]
V542 es : [4.72862847]
V543 es : [4.72812861]
V544 es : [4.72762875]
V545 es : [4.72712889]
V546 es : [4.72662903]
V547 es : [4.72612916]
V548 es : [4.7256293]
V549 es : [4.72512944]
V550 es : [4.72462958]
V551 es : [4.72412972]
V552 es : [4.72362986]
V553 es : [4.72313]
V554 es : [4.72263014]
V555 es : [4.72213028]
V556 es : [4.72163042]
V557 es : [4.72113056]
V558 es : [4.7206307]
V559 es : [4.72013084]
V560 es : [4.71963098]
V561 es : [4.71913112]
V562 es : [4.71863126]
V563 es : [4.7181314]
V564 es : [4.717631

V1145 es : [4.42721253]
V1146 es : [4.42671267]
V1147 es : [4.42621281]
V1148 es : [4.42571295]
V1149 es : [4.42521309]
V1150 es : [4.42471323]
V1151 es : [4.42421336]
V1152 es : [4.4237135]
V1153 es : [4.42321364]
V1154 es : [4.42271378]
V1155 es : [4.42221392]
V1156 es : [4.42171406]
V1157 es : [4.4212142]
V1158 es : [4.42071434]
V1159 es : [4.42021448]
V1160 es : [4.41971462]
V1161 es : [4.41921476]
V1162 es : [4.4187149]
V1163 es : [4.41821504]
V1164 es : [4.41771518]
V1165 es : [4.41721532]
V1166 es : [4.41671546]
V1167 es : [4.4162156]
V1168 es : [4.41571573]
V1169 es : [4.41521587]
V1170 es : [4.41471601]
V1171 es : [4.41421615]
V1172 es : [4.41371629]
V1173 es : [4.41321643]
V1174 es : [4.41271657]
V1175 es : [4.41221671]
V1176 es : [4.41171685]
V1177 es : [4.41121699]
V1178 es : [4.41071713]
V1179 es : [4.41021727]
V1180 es : [4.40971741]
V1181 es : [4.40921755]
V1182 es : [4.40871769]
V1183 es : [4.40821783]
V1184 es : [4.40771797]
V1185 es : [4.4072181]
V1186 es : [4.4067182

V1785 es : [4.10730175]
V1786 es : [4.10680189]
V1787 es : [4.10630203]
V1788 es : [4.10580217]
V1789 es : [4.1053023]
V1790 es : [4.10480244]
V1791 es : [4.10430258]
V1792 es : [4.10380272]
V1793 es : [4.10330286]
V1794 es : [4.102803]
V1795 es : [4.10230314]
V1796 es : [4.10180328]
V1797 es : [4.10130342]
V1798 es : [4.10080356]
V1799 es : [4.1003037]
V1800 es : [4.09980384]
V1801 es : [4.09930398]
V1802 es : [4.09880412]
V1803 es : [4.09830426]
V1804 es : [4.0978044]
V1805 es : [4.09730454]
V1806 es : [4.09680467]
V1807 es : [4.09630481]
V1808 es : [4.09580495]
V1809 es : [4.09530509]
V1810 es : [4.09480523]
V1811 es : [4.09430537]
V1812 es : [4.09380551]
V1813 es : [4.09330565]
V1814 es : [4.09280579]
V1815 es : [4.09230593]
V1816 es : [4.09180607]
V1817 es : [4.09130621]
V1818 es : [4.09080635]
V1819 es : [4.09030649]
V1820 es : [4.08980663]
V1821 es : [4.08930677]
V1822 es : [4.0888069]
V1823 es : [4.08830704]
V1824 es : [4.08780718]
V1825 es : [4.08730732]
V1826 es : [4.08680746

V2347 es : [3.82638009]
V2348 es : [3.82588023]
V2349 es : [3.82538037]
V2350 es : [3.82488051]
V2351 es : [3.82438065]
V2352 es : [3.82388079]
V2353 es : [3.82338093]
V2354 es : [3.82288107]
V2355 es : [3.82238121]
V2356 es : [3.82188135]
V2357 es : [3.82138149]
V2358 es : [3.82088163]
V2359 es : [3.82038176]
V2360 es : [3.8198819]
V2361 es : [3.81938204]
V2362 es : [3.81888218]
V2363 es : [3.81838232]
V2364 es : [3.81788246]
V2365 es : [3.8173826]
V2366 es : [3.81688274]
V2367 es : [3.81638288]
V2368 es : [3.81588302]
V2369 es : [3.81538316]
V2370 es : [3.8148833]
V2371 es : [3.81438344]
V2372 es : [3.81388358]
V2373 es : [3.81338372]
V2374 es : [3.81288386]
V2375 es : [3.812384]
V2376 es : [3.81188413]
V2377 es : [3.81138427]
V2378 es : [3.81088441]
V2379 es : [3.81038455]
V2380 es : [3.80988469]
V2381 es : [3.80938483]
V2382 es : [3.80888497]
V2383 es : [3.80838511]
V2384 es : [3.80788525]
V2385 es : [3.80738539]
V2386 es : [3.80688553]
V2387 es : [3.80638567]
V2388 es : [3.8058858

...

V7453 es : [1.27409189]
V7454 es : [1.27359203]
V7455 es : [1.27309217]
V7456 es : [1.27259231]
V7457 es : [1.27209245]
V7458 es : [1.27159258]
V7459 es : [1.27109272]
V7460 es : [1.27059286]
V7461 es : [1.270093]
V7462 es : [1.26959314]
V7463 es : [1.26909328]
V7464 es : [1.26859342]
V7465 es : [1.26809356]
V7466 es : [1.2675937]
V7467 es : [1.26709384]
V7468 es : [1.26659398]
V7469 es : [1.26609412]
V7470 es : [1.26559426]
V7471 es : [1.2650944]
V7472 es : [1.26459454]
V7473 es : [1.26409468]
V7474 es : [1.26359482]
V7475 es : [1.26309495]
V7476 es : [1.26259509]
V7477 es : [1.26209523]
V7478 es : [1.26159537]
V7479 es : [1.26109551]
V7480 es : [1.26059565]
V7481 es : [1.26009579]
V7482 es : [1.25959593]
V7483 es : [1.25909607]
V7484 es : [1.25859621]
V7485 es : [1.25809635]
V7486 es : [1.25759649]
V7487 es : [1.25709663]
V7488 es : [1.25659677]
V7489 es : [1.25609691]
V7490 es : [1.25559705]
V7491 es : [1.25509719]
V7492 es : [1.25459732]
V7493 es : [1.25409746]
V7494 es : [1.253597

V8026 es : [0.98767177]
V8027 es : [0.98717191]
V8028 es : [0.98667205]
V8029 es : [0.98617218]
V8030 es : [0.98567232]
V8031 es : [0.98517246]
V8032 es : [0.9846726]
V8033 es : [0.98417274]
V8034 es : [0.98367288]
V8035 es : [0.98317302]
V8036 es : [0.98267316]
V8037 es : [0.9821733]
V8038 es : [0.98167344]
V8039 es : [0.98117358]
V8040 es : [0.98067372]
V8041 es : [0.98017386]
V8042 es : [0.979674]
V8043 es : [0.97917414]
V8044 es : [0.97867428]
V8045 es : [0.97817441]
V8046 es : [0.97767455]
V8047 es : [0.97717469]
V8048 es : [0.97667483]
V8049 es : [0.97617497]
V8050 es : [0.97567511]
V8051 es : [0.97517525]
V8052 es : [0.97467539]
V8053 es : [0.97417553]
V8054 es : [0.97367567]
V8055 es : [0.97317581]
V8056 es : [0.97267595]
V8057 es : [0.97217609]
V8058 es : [0.97167623]
V8059 es : [0.97117637]
V8060 es : [0.97067651]
V8061 es : [0.97017665]
V8062 es : [0.96967678]
V8063 es : [0.96917692]
V8064 es : [0.96867706]
V8065 es : [0.9681772]
V8066 es : [0.96767734]
V8067 es : [0.9671774

V8616 es : [0.69275401]
V8617 es : [0.69225415]
V8618 es : [0.69175429]
V8619 es : [0.69125443]
V8620 es : [0.69075457]
V8621 es : [0.69025471]
V8622 es : [0.68975485]
V8623 es : [0.68925499]
V8624 es : [0.68875513]
V8625 es : [0.68825527]
V8626 es : [0.68775541]
V8627 es : [0.68725555]
V8628 es : [0.68675569]
V8629 es : [0.68625583]
V8630 es : [0.68575597]
V8631 es : [0.68525611]
V8632 es : [0.68475625]
V8633 es : [0.68425638]
V8634 es : [0.68375652]
V8635 es : [0.68325666]
V8636 es : [0.6827568]
V8637 es : [0.68225694]
V8638 es : [0.68175708]
V8639 es : [0.68125722]
V8640 es : [0.68075736]
V8641 es : [0.6802575]
V8642 es : [0.67975764]
V8643 es : [0.67925778]
V8644 es : [0.67875792]
V8645 es : [0.67825806]
V8646 es : [0.6777582]
V8647 es : [0.67725834]
V8648 es : [0.67675848]
V8649 es : [0.67625861]
V8650 es : [0.67575875]
V8651 es : [0.67525889]
V8652 es : [0.67475903]
V8653 es : [0.67425917]
V8654 es : [0.67375931]
V8655 es : [0.67325945]
V8656 es : [0.67275959]
V8657 es : [0.67225

V8961 es : [0.52030211]
V8962 es : [0.51980225]
V8963 es : [0.51930239]
V8964 es : [0.51880253]
V8965 es : [0.51830267]
V8966 es : [0.51780281]
V8967 es : [0.51730295]
V8968 es : [0.51680308]
V8969 es : [0.51630322]
V8970 es : [0.51580336]
V8971 es : [0.5153035]
V8972 es : [0.51480364]
V8973 es : [0.51430378]
V8974 es : [0.51380392]
V8975 es : [0.51330406]
V8976 es : [0.5128042]
V8977 es : [0.51230434]
V8978 es : [0.51180448]
V8979 es : [0.51130462]
V8980 es : [0.51080476]
V8981 es : [0.5103049]
V8982 es : [0.50980504]
V8983 es : [0.50930518]
V8984 es : [0.50880532]
V8985 es : [0.50830545]
V8986 es : [0.50780559]
V8987 es : [0.50730573]
V8988 es : [0.50680587]
V8989 es : [0.50630601]
V8990 es : [0.50580615]
V8991 es : [0.50530629]
V8992 es : [0.50480643]
V8993 es : [0.50430657]
V8994 es : [0.50380671]
V8995 es : [0.50330685]
V8996 es : [0.50280699]
V8997 es : [0.50230713]
V8998 es : [0.50180727]
V8999 es : [0.50130741]
V9000 es : [0.50080755]
V9001 es : [0.50030769]
V9002 es : [0.49980

V9645 es : [0.17839746]
V9646 es : [0.1778976]
V9647 es : [0.17739774]
V9648 es : [0.17689788]
V9649 es : [0.17639802]
V9650 es : [0.17589816]
V9651 es : [0.1753983]
V9652 es : [0.17489844]
V9653 es : [0.17439858]
V9654 es : [0.17389872]
V9655 es : [0.17339886]
V9656 es : [0.17289899]
V9657 es : [0.17239913]
V9658 es : [0.17189927]
V9659 es : [0.17139941]
V9660 es : [0.17089955]
V9661 es : [0.17039969]
V9662 es : [0.16989983]
V9663 es : [0.16939997]
V9664 es : [0.16890011]
V9665 es : [0.16840025]
V9666 es : [0.16790039]
V9667 es : [0.16740053]
V9668 es : [0.16690067]
V9669 es : [0.16640081]
V9670 es : [0.16590095]
V9671 es : [0.16540109]
V9672 es : [0.16490123]
V9673 es : [0.16440136]
V9674 es : [0.1639015]
V9675 es : [0.16340164]
V9676 es : [0.16290178]
V9677 es : [0.16240192]
V9678 es : [0.16190206]
V9679 es : [0.1614022]
V9680 es : [0.16090234]
V9681 es : [0.16040248]
V9682 es : [0.15990262]
V9683 es : [0.15940276]
V9684 es : [0.1589029]
V9685 es : [0.15840304]
V9686 es : [0.1579031

In [232]:
# con numpy 
# comprobando con numpy 
V = np.linalg.solve(A,b)
print("Respuesta es : \n")
for i in range(0,len(V)):
    print(f'V{i+1} es : {V[i]}')

Respuesta es : 

V1 es : [4.99888228]
V2 es : [4.99861842]
V3 es : [4.99802841]
V4 es : [4.99756299]
V5 es : [4.99704997]
V6 es : [4.99655514]
V7 es : [4.99605336]
V8 es : [4.99555423]
V9 es : [4.99505409]
V10 es : [4.99455433]
V11 es : [4.99405443]
V12 es : [4.99355459]
V13 es : [4.99305472]
V14 es : [4.99255486]
V15 es : [4.992055]
V16 es : [4.99155514]
V17 es : [4.99105528]
V18 es : [4.99055542]
V19 es : [4.99005556]
V20 es : [4.9895557]
V21 es : [4.98905584]
V22 es : [4.98855598]
V23 es : [4.98805612]
V24 es : [4.98755626]
V25 es : [4.9870564]
V26 es : [4.98655654]
V27 es : [4.98605667]
V28 es : [4.98555681]
V29 es : [4.98505695]
V30 es : [4.98455709]
V31 es : [4.98405723]
V32 es : [4.98355737]
V33 es : [4.98305751]
V34 es : [4.98255765]
V35 es : [4.98205779]
V36 es : [4.98155793]
V37 es : [4.98105807]
V38 es : [4.98055821]
V39 es : [4.98005835]
V40 es : [4.97955849]
V41 es : [4.97905863]
V42 es : [4.97855877]
V43 es : [4.97805891]
V44 es : [4.97755904]
V45 es : [4.97705918]
V46 es

V765 es : [4.61715956]
V766 es : [4.61665969]
V767 es : [4.61615983]
V768 es : [4.61565997]
V769 es : [4.61516011]
V770 es : [4.61466025]
V771 es : [4.61416039]
V772 es : [4.61366053]
V773 es : [4.61316067]
V774 es : [4.61266081]
V775 es : [4.61216095]
V776 es : [4.61166109]
V777 es : [4.61116123]
V778 es : [4.61066137]
V779 es : [4.61016151]
V780 es : [4.60966165]
V781 es : [4.60916179]
V782 es : [4.60866192]
V783 es : [4.60816206]
V784 es : [4.6076622]
V785 es : [4.60716234]
V786 es : [4.60666248]
V787 es : [4.60616262]
V788 es : [4.60566276]
V789 es : [4.6051629]
V790 es : [4.60466304]
V791 es : [4.60416318]
V792 es : [4.60366332]
V793 es : [4.60316346]
V794 es : [4.6026636]
V795 es : [4.60216374]
V796 es : [4.60166388]
V797 es : [4.60116402]
V798 es : [4.60066416]
V799 es : [4.60016429]
V800 es : [4.59966443]
V801 es : [4.59916457]
V802 es : [4.59866471]
V803 es : [4.59816485]
V804 es : [4.59766499]
V805 es : [4.59716513]
V806 es : [4.59666527]
V807 es : [4.59616541]
V808 es : [4.5

V1137 es : [4.43121141]
V1138 es : [4.43071155]
V1139 es : [4.43021169]
V1140 es : [4.42971183]
V1141 es : [4.42921197]
V1142 es : [4.42871211]
V1143 es : [4.42821225]
V1144 es : [4.42771239]
V1145 es : [4.42721253]
V1146 es : [4.42671267]
V1147 es : [4.42621281]
V1148 es : [4.42571295]
V1149 es : [4.42521309]
V1150 es : [4.42471323]
V1151 es : [4.42421336]
V1152 es : [4.4237135]
V1153 es : [4.42321364]
V1154 es : [4.42271378]
V1155 es : [4.42221392]
V1156 es : [4.42171406]
V1157 es : [4.4212142]
V1158 es : [4.42071434]
V1159 es : [4.42021448]
V1160 es : [4.41971462]
V1161 es : [4.41921476]
V1162 es : [4.4187149]
V1163 es : [4.41821504]
V1164 es : [4.41771518]
V1165 es : [4.41721532]
V1166 es : [4.41671546]
V1167 es : [4.4162156]
V1168 es : [4.41571573]
V1169 es : [4.41521587]
V1170 es : [4.41471601]
V1171 es : [4.41421615]
V1172 es : [4.41371629]
V1173 es : [4.41321643]
V1174 es : [4.41271657]
V1175 es : [4.41221671]
V1176 es : [4.41171685]
V1177 es : [4.41121699]
V1178 es : [4.410717

V1734 es : [4.13279464]
V1735 es : [4.13229478]
V1736 es : [4.13179492]
V1737 es : [4.13129506]
V1738 es : [4.13079519]
V1739 es : [4.13029533]
V1740 es : [4.12979547]
V1741 es : [4.12929561]
V1742 es : [4.12879575]
V1743 es : [4.12829589]
V1744 es : [4.12779603]
V1745 es : [4.12729617]
V1746 es : [4.12679631]
V1747 es : [4.12629645]
V1748 es : [4.12579659]
V1749 es : [4.12529673]
V1750 es : [4.12479687]
V1751 es : [4.12429701]
V1752 es : [4.12379715]
V1753 es : [4.12329729]
V1754 es : [4.12279743]
V1755 es : [4.12229756]
V1756 es : [4.1217977]
V1757 es : [4.12129784]
V1758 es : [4.12079798]
V1759 es : [4.12029812]
V1760 es : [4.11979826]
V1761 es : [4.1192984]
V1762 es : [4.11879854]
V1763 es : [4.11829868]
V1764 es : [4.11779882]
V1765 es : [4.11729896]
V1766 es : [4.1167991]
V1767 es : [4.11629924]
V1768 es : [4.11579938]
V1769 es : [4.11529952]
V1770 es : [4.11479966]
V1771 es : [4.1142998]
V1772 es : [4.11379993]
V1773 es : [4.11330007]
V1774 es : [4.11280021]
V1775 es : [4.112300

V2292 es : [3.85387242]
V2293 es : [3.85337256]
V2294 es : [3.8528727]
V2295 es : [3.85237284]
V2296 es : [3.85187298]
V2297 es : [3.85137312]
V2298 es : [3.85087326]
V2299 es : [3.8503734]
V2300 es : [3.84987354]
V2301 es : [3.84937368]
V2302 es : [3.84887382]
V2303 es : [3.84837396]
V2304 es : [3.8478741]
V2305 es : [3.84737424]
V2306 es : [3.84687438]
V2307 es : [3.84637452]
V2308 es : [3.84587466]
V2309 es : [3.84537479]
V2310 es : [3.84487493]
V2311 es : [3.84437507]
V2312 es : [3.84387521]
V2313 es : [3.84337535]
V2314 es : [3.84287549]
V2315 es : [3.84237563]
V2316 es : [3.84187577]
V2317 es : [3.84137591]
V2318 es : [3.84087605]
V2319 es : [3.84037619]
V2320 es : [3.83987633]
V2321 es : [3.83937647]
V2322 es : [3.83887661]
V2323 es : [3.83837675]
V2324 es : [3.83787689]
V2325 es : [3.83737702]
V2326 es : [3.83687716]
V2327 es : [3.8363773]
V2328 es : [3.83587744]
V2329 es : [3.83537758]
V2330 es : [3.83487772]
V2331 es : [3.83437786]
V2332 es : [3.833878]
V2333 es : [3.83337814

...

V7327 es : [1.33707432]
V7328 es : [1.33657446]
V7329 es : [1.3360746]
V7330 es : [1.33557474]
V7331 es : [1.33507488]
V7332 es : [1.33457502]
V7333 es : [1.33407516]
V7334 es : [1.3335753]
V7335 es : [1.33307544]
V7336 es : [1.33257558]
V7337 es : [1.33207572]
V7338 es : [1.33157586]
V7339 es : [1.331076]
V7340 es : [1.33057614]
V7341 es : [1.33007627]
V7342 es : [1.32957641]
V7343 es : [1.32907655]
V7344 es : [1.32857669]
V7345 es : [1.32807683]
V7346 es : [1.32757697]
V7347 es : [1.32707711]
V7348 es : [1.32657725]
V7349 es : [1.32607739]
V7350 es : [1.32557753]
V7351 es : [1.32507767]
V7352 es : [1.32457781]
V7353 es : [1.32407795]
V7354 es : [1.32357809]
V7355 es : [1.32307823]
V7356 es : [1.32257837]
V7357 es : [1.32207851]
V7358 es : [1.32157864]
V7359 es : [1.32107878]
V7360 es : [1.32057892]
V7361 es : [1.32007906]
V7362 es : [1.3195792]
V7363 es : [1.31907934]
V7364 es : [1.31857948]
V7365 es : [1.31807962]
V7366 es : [1.31757976]
V7367 es : [1.3170799]
V7368 es : [1.31658004

V7814 es : [1.09364221]
V7815 es : [1.09314235]
V7816 es : [1.09264249]
V7817 es : [1.09214263]
V7818 es : [1.09164277]
V7819 es : [1.09114291]
V7820 es : [1.09064305]
V7821 es : [1.09014319]
V7822 es : [1.08964333]
V7823 es : [1.08914347]
V7824 es : [1.08864361]
V7825 es : [1.08814375]
V7826 es : [1.08764389]
V7827 es : [1.08714402]
V7828 es : [1.08664416]
V7829 es : [1.0861443]
V7830 es : [1.08564444]
V7831 es : [1.08514458]
V7832 es : [1.08464472]
V7833 es : [1.08414486]
V7834 es : [1.083645]
V7835 es : [1.08314514]
V7836 es : [1.08264528]
V7837 es : [1.08214542]
V7838 es : [1.08164556]
V7839 es : [1.0811457]
V7840 es : [1.08064584]
V7841 es : [1.08014598]
V7842 es : [1.07964612]
V7843 es : [1.07914626]
V7844 es : [1.07864639]
V7845 es : [1.07814653]
V7846 es : [1.07764667]
V7847 es : [1.07714681]
V7848 es : [1.07664695]
V7849 es : [1.07614709]
V7850 es : [1.07564723]
V7851 es : [1.07514737]
V7852 es : [1.07464751]
V7853 es : [1.07414765]
V7854 es : [1.07364779]
V7855 es : [1.073147

V8385 es : [0.80822181]
V8386 es : [0.80772195]
V8387 es : [0.80722209]
V8388 es : [0.80672223]
V8389 es : [0.80622237]
V8390 es : [0.80572251]
V8391 es : [0.80522265]
V8392 es : [0.80472279]
V8393 es : [0.80422293]
V8394 es : [0.80372307]
V8395 es : [0.80322321]
V8396 es : [0.80272335]
V8397 es : [0.80222349]
V8398 es : [0.80172362]
V8399 es : [0.80122376]
V8400 es : [0.8007239]
V8401 es : [0.80022404]
V8402 es : [0.79972418]
V8403 es : [0.79922432]
V8404 es : [0.79872446]
V8405 es : [0.7982246]
V8406 es : [0.79772474]
V8407 es : [0.79722488]
V8408 es : [0.79672502]
V8409 es : [0.79622516]
V8410 es : [0.7957253]
V8411 es : [0.79522544]
V8412 es : [0.79472558]
V8413 es : [0.79422572]
V8414 es : [0.79372585]
V8415 es : [0.79322599]
V8416 es : [0.79272613]
V8417 es : [0.79222627]
V8418 es : [0.79172641]
V8419 es : [0.79122655]
V8420 es : [0.79072669]
V8421 es : [0.79022683]
V8422 es : [0.78972697]
V8423 es : [0.78922711]
V8424 es : [0.78872725]
V8425 es : [0.78822739]
V8426 es : [0.78772

V8946 es : [0.52780002]
V8947 es : [0.52730016]
V8948 es : [0.5268003]
V8949 es : [0.52630044]
V8950 es : [0.52580058]
V8951 es : [0.52530071]
V8952 es : [0.52480085]
V8953 es : [0.52430099]
V8954 es : [0.52380113]
V8955 es : [0.52330127]
V8956 es : [0.52280141]
V8957 es : [0.52230155]
V8958 es : [0.52180169]
V8959 es : [0.52130183]
V8960 es : [0.52080197]
V8961 es : [0.52030211]
V8962 es : [0.51980225]
V8963 es : [0.51930239]
V8964 es : [0.51880253]
V8965 es : [0.51830267]
V8966 es : [0.51780281]
V8967 es : [0.51730295]
V8968 es : [0.51680308]
V8969 es : [0.51630322]
V8970 es : [0.51580336]
V8971 es : [0.5153035]
V8972 es : [0.51480364]
V8973 es : [0.51430378]
V8974 es : [0.51380392]
V8975 es : [0.51330406]
V8976 es : [0.5128042]
V8977 es : [0.51230434]
V8978 es : [0.51180448]
V8979 es : [0.51130462]
V8980 es : [0.51080476]
V8981 es : [0.5103049]
V8982 es : [0.50980504]
V8983 es : [0.50930518]
V8984 es : [0.50880532]
V8985 es : [0.50830545]
V8986 es : [0.50780559]
V8987 es : [0.507305

V9519 es : [0.2413799]
V9520 es : [0.24088004]
V9521 es : [0.24038018]
V9522 es : [0.23988031]
V9523 es : [0.23938045]
V9524 es : [0.23888059]
V9525 es : [0.23838073]
V9526 es : [0.23788087]
V9527 es : [0.23738101]
V9528 es : [0.23688115]
V9529 es : [0.23638129]
V9530 es : [0.23588143]
V9531 es : [0.23538157]
V9532 es : [0.23488171]
V9533 es : [0.23438185]
V9534 es : [0.23388199]
V9535 es : [0.23338213]
V9536 es : [0.23288227]
V9537 es : [0.23238241]
V9538 es : [0.23188254]
V9539 es : [0.23138268]
V9540 es : [0.23088282]
V9541 es : [0.23038296]
V9542 es : [0.2298831]
V9543 es : [0.22938324]
V9544 es : [0.22888338]
V9545 es : [0.22838352]
V9546 es : [0.22788366]
V9547 es : [0.2273838]
V9548 es : [0.22688394]
V9549 es : [0.22638408]
V9550 es : [0.22588422]
V9551 es : [0.22538436]
V9552 es : [0.2248845]
V9553 es : [0.22438464]
V9554 es : [0.22388478]
V9555 es : [0.22338491]
V9556 es : [0.22288505]
V9557 es : [0.22238519]
V9558 es : [0.22188533]
V9559 es : [0.22138547]
V9560 es : [0.220885